# Jane Street Market Prediction
[Kaggle Jane Street Market Prediction](https://www.kaggle.com/c/jane-street-market-prediction/overview)


In [2]:
import pandas as pd
import numpy as np
import datatable as dt
X = dt.fread('jane-street-market-prediction/train.csv').to_pandas()

In [3]:
Xt = X[X.date >= 460]
yt = np.where((Xt.resp > 0), 1, 0)
X = X[(X.date > 85) & (X.date < 460)]
y = np.where((X.resp > 0), 1, 0)
drop_cols = ['date', 'resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp', 'ts_id']
X.drop(drop_cols, axis=1, inplace=True)
Xt.drop(drop_cols, axis=1, inplace=True)

In [4]:
import lightgbm as lgb
lgb_train = lgb.Dataset(X, y)
lgb_eval = lgb.Dataset(Xt, yt, reference=lgb_train)


In [5]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [6]:
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 3.112413 seconds.
You can set `force_col_wise=true` to remove the overhead.
[1]	valid_0's l2: 0.249993	valid_0's l1: 0.499981
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l2: 0.249952	valid_0's l1: 0.499937
[3]	valid_0's l2: 0.249913	valid_0's l1: 0.499892
[4]	valid_0's l2: 0.249874	valid_0's l1: 0.499845
[5]	valid_0's l2: 0.249842	valid_0's l1: 0.499803
[6]	valid_0's l2: 0.249804	valid_0's l1: 0.499754
[7]	valid_0's l2: 0.249797	valid_0's l1: 0.499736
[8]	valid_0's l2: 0.249768	valid_0's l1: 0.499695
[9]	valid_0's l2: 0.249748	valid_0's l1: 0.499664
[10]	valid_0's l2: 0.249729	valid_0's l1: 0.499633
[11]	valid_0's l2: 0.249712	valid_0's l1: 0.499608
[12]	valid_0's l2: 0.249687	valid_0's l1: 0.499573
[13]	valid_0's l2: 0.249671	valid_0's l1: 0.499548
[14]	valid_0's l2: 0.249651	valid_0's l1: 0.499515
[15]	valid_0's l2: 0.249627	valid_0's l1: 0.499479
[16]	valid_0's l2: 

In [7]:
y_pred = gbm.predict(Xt, num_iteration=gbm.best_iteration)

In [10]:
y_pred

array([0.50256086, 0.50659055, 0.50026269, ..., 0.50093084, 0.5131895 ,
       0.50151323])

In [9]:
from sklearn.metrics import mean_squared_error
print('The rmse of prediction is:', mean_squared_error(yt, y_pred) ** 0.5)

The rmse of prediction is: 0.49952223817392444
